### LCEL
LCEL (LangChain Expression Language) is a concise, chainable syntax for composing LangChain components like prompts, models, retrievers, and output parsers in a pipeline. It simplifies building modular, readable, and reusable workflows for language model applications.

In [33]:
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
# Loads environment variables from a .env file into your system's environment.
load_dotenv()
# os.environ is a dictionary in Python that holds all the environment variables available to your system during the program's execution.

# This is needed to enable LangSmith logging/tracing for chains, prompts, and LLM calls.
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

# Setting this to 'true' will start recording all your chain/agent/LLM activity for debugging and evaluation.
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

# This helps organize and filter traces inside the LangSmith dashboard based on this project.
os.environ['Langchain_Project'] = os.getenv('LANGCHAIN_PROJECT')

In [34]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [35]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(api_key=groq_api_key, model="Gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A2FE8E4E90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A2FE8F4590>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

##### Without Using LCEL to inference LLM, Prompt and Stroutput Chains

In [36]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

response = model.invoke(messages)
print(response)

content="Hello How are you? in French is:\n\n**Bonjour, comment allez-vous ?** \n\n\nLet me know if you have any other phrases you'd like translated!\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 21, 'total_tokens': 60, 'completion_time': 0.070909091, 'prompt_time': 0.002119957, 'queue_time': 0.233693447, 'total_time': 0.073029048}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-e7e5d459-163d-4112-9a50-de5558be877f-0' usage_metadata={'input_tokens': 21, 'output_tokens': 39, 'total_tokens': 60}


In [37]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
output_response = parser.invoke(response)
print(output_response)

Hello How are you? in French is:

**Bonjour, comment allez-vous ?** 


Let me know if you have any other phrases you'd like translated!



##### With Using LCEL to inference LLM, Prompt and Stroutput Chains

In [38]:
chain = (model|parser).invoke(messages)
chain

'Here are a few ways to translate "Hello, How are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, comment ça va ?** (This is a more casual way to greet someone you know well.)\n* **Coucou, comment vas-tu ?** (This is a very informal greeting, often used with friends or family.)\n\nThe best option depends on the context and your relationship with the person you are greeting. \n\n\n\n'

#### Using PromptTemplate with LCEL : An efficient technique of LCEL

In [39]:
from langchain_core.prompts import ChatPromptTemplate

template = "You are a French language tutor. Translate the following sentence in {language}:"

prompt = ChatPromptTemplate(
    [
        ("system", template),
        ("user", "{text}")
    ]
)

In [40]:
chain = prompt | model | parser
result = chain.invoke({"language":"French", "text":"Hello How are you?"})
result

'There are two ways to translate "Hello, how are you?" in French, depending on the level of formality:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a more casual way to greet someone you know well.)\n\n\nLet me know if you\'d like to learn more French greetings! 😊\n\n'

In [41]:
print(result)

There are two ways to translate "Hello, how are you?" in French, depending on the level of formality:

**Formal:**

* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)

**Informal:**

* **Salut, comment vas-tu ?** (This is a more casual way to greet someone you know well.)


Let me know if you'd like to learn more French greetings! 😊


